Robbe Van Herpe groep 30 analyse 3

1. Imports

In [1]:
import pandas as pd
import os
import pyodbc
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import classification_report

2. Inladen data van DWH

In [2]:
db_host = os.getenv('DB_HOST', 'localhost')
db_name = os.getenv('DB_NAME', 'DEP_DWH_G30')

connection_string = f'DRIVER={{ODBC Driver 18 for SQL Server}};SERVER={db_host};DATABASE={db_name};Trusted_Connection=yes;Encrypt=no;TrustServerCertificate=yes'
connection = pyodbc.connect(connection_string)

klassement = pd.read_sql("SELECT * FROM FactKlassement", connection)
datum = pd.read_sql("SELECT * FROM DimDate", connection)

C:\Users\Robbe\AppData\Local\Temp\ipykernel_2192\1695276803.py:7: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  klassement = pd.read_sql("SELECT * FROM FactKlassement", connection)
C:\Users\Robbe\AppData\Local\Temp\ipykernel_2192\1695276803.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  datum = pd.read_sql("SELECT * FROM DimDate", connection)


3. Bekijken data

In [3]:
klassement.head()

,KlassementKey,BeginDateKey,EindeDateKey,TeamKey,Stand,AantalGespeeld,AantalGewonnen,AantalGelijk,AantalVerloren,DoelpuntenVoor,DoelpuntenTegen,DoelpuntenSaldo,PuntenVoor2ptn,PuntenTegen2ptn,PuntenVoor3ptn
0,1,141,147,28,13,18,6,4,8,14,23,-9,16,20,22
1,2,92,98,28,14,11,2,2,7,7,20,-13,6,16,8
2,3,169,175,28,13,22,7,5,10,19,29,-10,19,25,26
3,4,106,112,43,2,13,7,4,2,28,12,16,18,8,25
4,5,127,133,28,14,16,5,3,8,13,23,-10,13,19,18


In [4]:
klassement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35898 entries, 0 to 35897
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   KlassementKey    35898 non-null  int64
 1   BeginDateKey     35898 non-null  int64
 2   EindeDateKey     35898 non-null  int64
 3   TeamKey          35898 non-null  int64
 4   Stand            35898 non-null  int64
 5   AantalGespeeld   35898 non-null  int64
 6   AantalGewonnen   35898 non-null  int64
 7   AantalGelijk     35898 non-null  int64
 8   AantalVerloren   35898 non-null  int64
 9   DoelpuntenVoor   35898 non-null  int64
 10  DoelpuntenTegen  35898 non-null  int64
 11  DoelpuntenSaldo  35898 non-null  int64
 12  PuntenVoor2ptn   35898 non-null  int64
 13  PuntenTegen2ptn  35898 non-null  int64
 14  PuntenVoor3ptn   35898 non-null  int64
dtypes: int64(15)
memory usage: 4.1 MB


In [5]:
datum.head()

,DateKey,Datum,Seizoen,Speeldag,DagVanDeMaand,DagVanHetJaar,WeekVanHetJaar,DagVanDeWeek,Maand,Semester,Kwartaal,Jaar,EngelseDag,EngelseMaand,DDMMJJJJ,IsWeekend,NaamDagVanDeWeek,IsSchrikkeljaar,WeekVanDeMaand,Tijdvak
0,1,1960-09-04,1960,1,4,248,37,6,9,2,3,1960,Sunday,September,04091960,True,Zondag,True,2,Herfst
1,2,1960-09-05,1960,1,5,249,37,0,9,2,3,1960,Monday,September,05091960,False,Maandag,True,2,Herfst
2,3,1960-09-06,1960,1,6,250,37,1,9,2,3,1960,Tuesday,September,06091960,False,Dinsdag,True,2,Herfst
3,4,1960-09-07,1960,1,7,251,37,2,9,2,3,1960,Wednesday,September,07091960,False,Woensdag,True,2,Herfst
4,5,1960-09-08,1960,1,8,252,37,3,9,2,3,1960,Thursday,September,08091960,False,Donderdag,True,2,Herfst


In [6]:
datum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23250 entries, 0 to 23249
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   DateKey           23250 non-null  int64 
 1   Datum             23250 non-null  object
 2   Seizoen           23250 non-null  object
 3   Speeldag          23250 non-null  int64 
 4   DagVanDeMaand     23250 non-null  int64 
 5   DagVanHetJaar     23250 non-null  int64 
 6   WeekVanHetJaar    23250 non-null  int64 
 7   DagVanDeWeek      23250 non-null  int64 
 8   Maand             23250 non-null  int64 
 9   Semester          23250 non-null  int64 
 10  Kwartaal          23250 non-null  int64 
 11  Jaar              23250 non-null  int64 
 12  EngelseDag        23250 non-null  object
 13  EngelseMaand      23250 non-null  object
 14  DDMMJJJJ          23250 non-null  object
 15  IsWeekend         23250 non-null  bool  
 16  NaamDagVanDeWeek  23250 non-null  object
 17  IsSchrikkelj

4. Data herwerken voor model

In [7]:
datum = datum[['DateKey','Seizoen']]

In [8]:
klassement = pd.merge(klassement, datum, left_on='BeginDateKey', right_on='DateKey', how='inner') 

In [9]:
klassement.head()

,KlassementKey,BeginDateKey,EindeDateKey,TeamKey,Stand,AantalGespeeld,AantalGewonnen,AantalGelijk,AantalVerloren,DoelpuntenVoor,DoelpuntenTegen,DoelpuntenSaldo,PuntenVoor2ptn,PuntenTegen2ptn,PuntenVoor3ptn,DateKey,Seizoen
0,1,141,147,28,13,18,6,4,8,14,23,-9,16,20,22,141,1960
1,2,92,98,28,14,11,2,2,7,7,20,-13,6,16,8,92,1960
2,3,169,175,28,13,22,7,5,10,19,29,-10,19,25,26,169,1960
3,4,106,112,43,2,13,7,4,2,28,12,16,18,8,25,106,1960
4,5,127,133,28,14,16,5,3,8,13,23,-10,13,19,18,127,1960


In [10]:
klassement = klassement[['KlassementKey','Stand', 'Seizoen', 'TeamKey', 'AantalGespeeld', 'AantalGewonnen', 'AantalGelijk', 'AantalVerloren', 'DoelpuntenVoor', 'DoelpuntenTegen', 'PuntenVoor2ptn', 'PuntenTegen2ptn', 'PuntenVoor3ptn']]

In [11]:
klassement.head()

,KlassementKey,Stand,Seizoen,TeamKey,AantalGespeeld,AantalGewonnen,AantalGelijk,AantalVerloren,DoelpuntenVoor,DoelpuntenTegen,PuntenVoor2ptn,PuntenTegen2ptn,PuntenVoor3ptn
0,1,13,1960,28,18,6,4,8,14,23,16,20,22
1,2,14,1960,28,11,2,2,7,7,20,6,16,8
2,3,13,1960,28,22,7,5,10,19,29,19,25,26
3,4,2,1960,43,13,7,4,2,28,12,18,8,25
4,5,14,1960,28,16,5,3,8,13,23,13,19,18


In [12]:
klassement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35898 entries, 0 to 35897
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   KlassementKey    35898 non-null  int64 
 1   Stand            35898 non-null  int64 
 2   Seizoen          35898 non-null  object
 3   TeamKey          35898 non-null  int64 
 4   AantalGespeeld   35898 non-null  int64 
 5   AantalGewonnen   35898 non-null  int64 
 6   AantalGelijk     35898 non-null  int64 
 7   AantalVerloren   35898 non-null  int64 
 8   DoelpuntenVoor   35898 non-null  int64 
 9   DoelpuntenTegen  35898 non-null  int64 
 10  PuntenVoor2ptn   35898 non-null  int64 
 11  PuntenTegen2ptn  35898 non-null  int64 
 12  PuntenVoor3ptn   35898 non-null  int64 
dtypes: int64(12), object(1)
memory usage: 3.6+ MB


In [13]:
klassement['Seizoen'] = klassement['Seizoen'].astype(int)

In [14]:
klassement.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35898 entries, 0 to 35897
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype
---  ------           --------------  -----
 0   KlassementKey    35898 non-null  int64
 1   Stand            35898 non-null  int64
 2   Seizoen          35898 non-null  int32
 3   TeamKey          35898 non-null  int64
 4   AantalGespeeld   35898 non-null  int64
 5   AantalGewonnen   35898 non-null  int64
 6   AantalGelijk     35898 non-null  int64
 7   AantalVerloren   35898 non-null  int64
 8   DoelpuntenVoor   35898 non-null  int64
 9   DoelpuntenTegen  35898 non-null  int64
 10  PuntenVoor2ptn   35898 non-null  int64
 11  PuntenTegen2ptn  35898 non-null  int64
 12  PuntenVoor3ptn   35898 non-null  int64
dtypes: int32(1), int64(12)
memory usage: 3.4 MB


5. Maken van Training en test data

In [15]:
klassement_data = klassement[['KlassementKey', 'Seizoen', 'TeamKey', 'AantalGespeeld', 'AantalGewonnen', 'AantalGelijk', 'AantalVerloren', 'DoelpuntenVoor', 'DoelpuntenTegen', 'PuntenVoor2ptn', 'PuntenTegen2ptn', 'PuntenVoor3ptn']] 
Stand_data = klassement['Stand'] 


klassement_train, klassement_test, Stand_train, Stand_test = train_test_split(klassement_data, Stand_data, test_size=0.20, random_state=42)


In [16]:
#Tonen records in test set
print(f"Number of records in training set: {klassement_train.shape[0]}")
print(f"Number of records in testing set: {klassement_test.shape[0]}")


Number of records in training set: 28718
Number of records in testing set: 7180


6. Kiezen en Testen van model
 
   We kiezen voor Classification want dit is het beste voor categorische data zoals een klassement voorspellen. 
   Deze is ook goed met imputs als win/verlies/gelijkspel en daardoor perfect voor mijn model.

   Vervolgens een model kiezen het model dat het beste was was randomForest maar deze was al genomen door een teamgenoot,
   dus heb ik gekozen voor Logistic Regression deze is goed met minder complexe relaties en zeer performant maar minder optimaal als randomForest voor deze taak.

In [19]:
# Model LogisticRegression uitvoeren

model = make_pipeline(
    StandardScaler(),
    LogisticRegression(multi_class='multinomial', solver='lbfgs', C=1, max_iter=200, random_state=42)
)

# Fit het model
model.fit(klassement_train, Stand_train)

# Predict aan de hand van de test data
y_pred = model.predict(klassement_test)

mae = mean_absolute_error(Stand_test, y_pred)
print("MAE van het model :", mae)

MAE van het model : 0.9944289693593314


7. Predictie van model vergelijken

In [18]:
y_pred = model.predict(klassement_test)
result = pd.DataFrame()
y_test = Stand_test.reset_index(drop=True)

# Aanmaken van kolommen en uitprinten
result['Voorspeld'] = y_pred.round()
result['Echte waarde'] = y_test
result[['Voorspeld','Echte waarde']].head()

,Voorspeld,Echte waarde
0,8,9
1,7,8
2,16,15
3,1,1
4,11,11
